# Base case of graph classification

* default hp setting same as utils.py
* epoch = 500
* 1000genome: train accuracy: 0.945, val acc: 0.907, test acc: 0.923 


In [1]:
# imports
import os.path as osp
import random
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score)
from sklearn.model_selection import train_test_split
from torch.distributions import LogNormal
from torch.nn import (CrossEntropyLoss, Linear, ModuleList, ReLU, Sequential,
                      Tanh)
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.loader import (DataLoader, ImbalancedSampler,
                                    NeighborLoader)
from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.nn import global_mean_pool as gap
from tqdm import tqdm

from psd_gnn.dataset import Merge_PSD_Dataset, PSD_Dataset
from psd_gnn.models.graph_classifier import GNN
# from psd_gnn.models.graph_classifier import GNN
from psd_gnn.utils import process_args, eval_metrics, create_dir

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)

In [2]:
# Global settings

if torch.backends.mps.is_available():
    DEVICE = torch.device("cpu")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda:0")
else:
    DEVICE = torch.device("cpu")
print(DEVICE)

cpu


In [3]:
# Model definitions
wf = "1000genome"


# ROOT = osp.join(osp.expanduser("~"), "tmp", "data", wf)
dataset = PSD_Dataset("./", wf, binary_labels=True, node_level=False)
n_graphs = len(dataset)
train_idx, test_idx = train_test_split(np.arange(n_graphs), test_size=0.6, random_state=0, shuffle=True)
val_idx, test_idx = train_test_split(test_idx, test_size=0.5, random_state=0, shuffle=True)

train_loader = DataLoader(dataset[train_idx], batch_size=32)
val_loader = DataLoader(dataset[val_idx], batch_size=32)
test_loader = DataLoader(dataset[test_idx], batch_size=32)

NUM_NODE_FEATURES = dataset.num_node_features
NUM_OUT_FEATURES = dataset.num_classes

model = GNN(NUM_NODE_FEATURES, 64, NUM_OUT_FEATURES).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# NOTE: ignore the weighted loss for now
loss_func = CrossEntropyLoss()

In [4]:
def train(model, loader):
    """ Train function

    Args:
        model (object): GNN model instance.
        loader (pyg.loader.DataLoader): Data loader object.

    Returns:
        float: Training accuracy.
    """
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(DEVICE)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = loss_func(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(loader.dataset)


@torch.no_grad()
def test(model, loader):
    """ Evaluation function.

    Args:
        model (object): GNN model instance.
        loader (pyg.loader.DataLoader): Data loader object.

    Returns:
        tuple (float, list): Testing accuracy, predicted labels.
    """
    model.eval()

    total_correct = 0
    y_pred = []
    y_true = []
    for data in loader:
        data = data.to(DEVICE)
        pred = model(data.x, data.edge_index, data.batch).argmax(dim=-1)
        # aggregate the y_pred from batches
        y_pred += pred.detach().cpu().numpy().tolist()
        y_true += data.y.detach().cpu().numpy().tolist()
        total_correct += int((pred == data.y).sum())

    return total_correct / len(loader.dataset), y_pred, y_true, 0

In [5]:
''' train and eval model '''
# NOTE: increase the epochs to 500. The same as the settings in IJHPCA paper
workflows = ["1000genome", 
        "nowcast-clustering-8",
        "nowcast-clustering-16",
        "wind-clustering-casa",
        "wind-noclustering-casa"]
''' Build GNN model '''
from torch.utils.tensorboard import SummaryWriter
for wf in  workflows:
    # ROOT = osp.join(osp.expanduser("~"), "tmp", "data", wf)
    dataset = PSD_Dataset("./", wf, binary_labels=True, node_level=False)
    n_graphs = len(dataset)
    train_idx, test_idx = train_test_split(np.arange(n_graphs), test_size=0.6, random_state=0, shuffle=True)
    val_idx, test_idx = train_test_split(test_idx, test_size=0.5, random_state=0, shuffle=True)

    train_loader = DataLoader(dataset[train_idx], batch_size=32)
    val_loader = DataLoader(dataset[val_idx], batch_size=32)
    test_loader = DataLoader(dataset[test_idx], batch_size=32)

    NUM_NODE_FEATURES = dataset.num_node_features
    NUM_OUT_FEATURES = dataset.num_classes

    model = GNN(NUM_NODE_FEATURES, 64, NUM_OUT_FEATURES).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    # NOTE: ignore the weighted loss for now
    loss_func = CrossEntropyLoss()

    loss_array =[]
    acc_arr=[]
    f1_arr=[]
    precision_arr=[]
    recall_arr=[]

    ts_start = datetime.now().strftime('%Y%m%d_%H%M%S')
    pbar = tqdm(range(1000), desc=wf)
    best = 0

    for e in pbar:
        train_loss = train(model, train_loader)
        train_acc, y_pred, _, _ = test(model, train_loader)
        val_acc, _, _,_ = test(model, val_loader)
        test_acc, y_pred, y_true, _ = test(model, test_loader)


        if val_acc > best:
            # NOTE: ignore of model dumping for now
            # save tmp model on disk
            # create_dir("tmp_models")
            # torch.save(model, osp.join(f"tmp_models",
            #                             f"saved_models_{args['workflow']}_{ts_start}"))
            best = val_acc





        # scheduler.step(train_loss) 
        ## Record all info
        loss_array.append(train_loss)
        conf_mat = confusion_matrix(y_true, y_pred)
        prec_val = precision_score(y_true, y_pred)
        f1_val = f1_score(y_true, y_pred)
        recall_val = recall_score(y_true, y_pred)
        pbar.set_postfix({"train_loss": train_loss,\
                        "train_acc": train_acc,\
                        "test_acc": test_acc,\
                        "f1 score": f1_val, "val_best":best})
        acc_arr.append(test_acc)
        f1_arr.append(f1_val)
        precision_arr.append(prec_val)
        recall_arr.append(recall_val)

        if e % 100 == 0:
        #     # ## The resulting information
        #     # actual_Data = model.dist.sample_n( 10000 )
        #     # probs = model.dist.log_prob(actual_Data).exp().squeeze(1)
        #     # args = np.argsort(actual_Data.squeeze(1).detach().cpu().numpy() )
        #     # probs = probs[args]
        #     # actual_Data = actual_Data[args]
        #     # train_acc, y_pred, y_true, outputs = test(model, train_loader)
        #     # plt.plot(actual_Data, probs)
        #     # x, bins, p = plt.hist(outputs, density=True)
        #     # for item in p:
        #     #     item.set_height(item.get_height()/sum(x))
        #     # # plt.xlim([0,1])
        #     # # plt.ylim([0,1])
        #     # plt.title(wfs)
        #     # # plt.savefig('CL_Figures/'+wfs+'prob'+'.png', dpi = 500)
        #     # plt.show()
        #     # ## The final training metrics
        #     # ys = []    
        # test_acc, y_pred, y_true, _ = test(model, test_loader)
        # print(np.unique(y_pred, return_counts=True), "real", np.unique(y_true, return_counts=True) )
        # conf_mat = confusion_matrix(y_true, y_pred)
        # prec_val = precision_score(y_true, y_pred)
        # f1_val = f1_score(y_true, y_pred)
        # recall_val = recall_score(y_true, y_pred)
            print("graph level clf:", wf,
                f"binary True",
                f"test acc {test_acc:.4f}",
                f"f1 {f1_val:.4f}",
                f"recall {recall_val:.4f}",
                f"prec {prec_val:.4f}", conf_mat, "\n"
                )
            

    np.savetxt('CL_res/'+wf+'upper_training.csv', np.array(loss_array), delimiter = ',')
    np.savetxt('CL_res/'+wf+'upper_training_loss.csv', np.array(loss_array), delimiter = ',')
    np.savetxt('CL_res/'+wf+'upper_acc.csv', np.array(acc_arr), delimiter = ',')
    np.savetxt('CL_res/'+wf+'upper_precision.csv', np.array(precision_arr), delimiter = ',')
    np.savetxt('CL_res/'+wf+'upper_recall.csv', np.array(recall_arr), delimiter = ',')
    
    del model
    del optimizer
    del loss_func

1000genome:   0%|          | 2/1000 [00:00<03:30,  4.73it/s, train_loss=0.739, train_acc=0.152, test_acc=0.15, f1 score=0, val_best=0.171]

graph level clf: 1000genome binary True test acc 0.1496 f1 0.0000 recall 0.0000 prec 0.0000 [[ 57   0]
 [324   0]] 



1000genome:  10%|█         | 102/1000 [00:19<02:51,  5.25it/s, train_loss=0.404, train_acc=0.848, test_acc=0.85, f1 score=0.919, val_best=0.829]

graph level clf: 1000genome binary True test acc 0.8504 f1 0.9191 recall 1.0000 prec 0.8504 [[  0  57]
 [  0 324]] 



1000genome:  20%|██        | 202/1000 [00:38<02:33,  5.19it/s, train_loss=0.343, train_acc=0.848, test_acc=0.85, f1 score=0.919, val_best=0.829]

...

wind-noclustering-casa:  80%|████████  | 802/1000 [01:28<00:21,  9.21it/s, train_loss=0.4, train_acc=0.806, test_acc=0.773, f1 score=0.868, val_best=0.796]  

graph level clf: wind-noclustering-casa binary True test acc 0.7731 f1 0.8679 recall 0.9603 prec 0.7917 [[ 10  70]
 [ 11 266]] 



wind-noclustering-casa:  90%|█████████ | 902/1000 [01:39<00:10,  9.21it/s, train_loss=0.381, train_acc=0.815, test_acc=0.779, f1 score=0.87, val_best=0.812] 

graph level clf: wind-noclustering-casa binary True test acc 0.7787 f1 0.8699 recall 0.9531 prec 0.8000 [[ 14  66]
 [ 13 264]] 



wind-noclustering-casa: 100%|██████████| 1000/1000 [01:50<00:00,  9.09it/s, train_loss=0.365, train_acc=0.817, test_acc=0.798, f1 score=0.88, val_best=0.815]
